In [47]:
import nltk 
import pandas as pd
import sys
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.api import TokenizerI
from nltk.tokenize.toktok import ToktokTokenizer
toktok = ToktokTokenizer()


#reload(sys)
#sys.setdefaultencoding('utf-8')

In [48]:
data = pd.io.parsers.read_csv("https://raw.githubusercontent.com/Galanopoulog/DATA620-Assignment-6/master/Self-Driving-Cars.csv")
data = data.drop(data.columns[[1,2,3,4,7,8,9]], axis=1)
data.text.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True) #remove ascii characters from text
data.head(11)

,_unit_id,sentiment,sentiment:confidence,text
0,724227031,5,0.7579,Two places I'd invest all my money if I could:...
1,724227032,5,0.8775,Awesome! Google driverless cars will help the ...
2,724227033,2,0.6805,If Google maps can't keep up with road constru...
3,724227034,2,0.8820,Autonomous cars seem way overhyped given the t...
4,724227035,3,1.0000,Just saw Google self-driving car on I-34. It w...
5,724227036,3,1.0000,Will driverless cars eventually replace taxi d...
6,724227037,not_relevant,0.5367,Chicago metro expected to be fully autonomous ...
7,724227038,not_relevant,0.6548,I love the infotainment system in my new car. ...
8,724227039,5,0.7187,Autonomous vehicles could reduce traffic fatal...
9,724227040,1,0.6412,Driverless cars are not worth the risk. Don't...


In [49]:
len(data)
print len(data)

7156


In [50]:
# Count relevant and irrelevant
rel_count = len(data[data.sentiment!="not_relevant"])
irrel_count = len(data[data.sentiment=="not_relevant"])
neut_count = len(data[data.sentiment=="3"])

print "Relevant:",rel_count
print "Not_Relevant: ", irrel_count
print "Neutral: ", neut_count

Relevant: 6943
Not_Relevant:  213
Neutral:  4245


In [51]:
# What sizes should our sets be?
train_set = int(0.7 * len(data))
dev_set = int(.15 * len(data))
final_set = len(data) - train_set - dev_set
Total = train_set + dev_set + final_set

print "Training set: %d" %train_set
print "Evaluation set: %d" %dev_set
print "Testing set: %d" %final_set
print "Total: %d" %Total

Training set: 5009
Evaluation set: 1073
Testing set: 1074
Total: 7156


In [52]:

# Let's split our data into relevant and non-relevant parts.
relevant = data[data.sentiment!="not_relevant"]
nonrelevant = data[data.sentiment=="not_relevant"]

# Let's take 70%, 15% and the 15% of each
rel_train = int(0.7 * len(relevant))
rel_dev = int(.15 * len(relevant))
rel_fin = len(relevant) - rel_train - rel_dev
rel_Total = rel_train + rel_dev + rel_fin

irrel_train = int(0.7 * len(nonrelevant))
irrel_dev = int(.15 * len(nonrelevant))
irrel_fin = len(nonrelevant) - irrel_train - irrel_dev
irrel_Total = irrel_train + irrel_dev + irrel_fin

print "Training set: %d" %rel_train
print "Evaluation set: %d" %rel_dev
print "Testing set: %d" %rel_fin
print "Total: %d" %rel_Total
print "----------------------"
print "Training set: %d" %irrel_train
print "Evaluation set: %d" %irrel_dev
print "Testing set: %d" %irrel_fin
print "Total: %d" %irrel_Total

Training set: 4860
Evaluation set: 1041
Testing set: 1042
Total: 6943
----------------------
Training set: 149
Evaluation set: 31
Testing set: 33
Total: 213


In [53]:
# Make sets for relevant data
rel_train = relevant[0:4860]
rel_dev = relevant[4860:5901]
rel_fin = relevant[5901:6943]

# Make sets for non-relevant data
irrel_train = nonrelevant[0:149]
irrel_dev = nonrelevant[149:180]
irrel_fin = nonrelevant[180:231]

# Combine sets by set types
train_set = pd.concat([rel_train, irrel_train], axis=0)
dev_set = pd.concat([rel_dev, irrel_dev], axis=0)
final_set = pd.concat([rel_fin, irrel_fin], axis=0)

# Determine lengths and compare to set size estimates
print "Training set: %d" %len(train_set)
print "Evaluation set: %d" %len(dev_set)

train_set[0:10]

Training set: 5009
Evaluation set: 1072


,_unit_id,sentiment,sentiment:confidence,text
0,724227031,5,0.7579,Two places I'd invest all my money if I could:...
1,724227032,5,0.8775,Awesome! Google driverless cars will help the ...
2,724227033,2,0.6805,If Google maps can't keep up with road constru...
3,724227034,2,0.8820,Autonomous cars seem way overhyped given the t...
4,724227035,3,1.0000,Just saw Google self-driving car on I-34. It w...
5,724227036,3,1.0000,Will driverless cars eventually replace taxi d...
8,724227039,5,0.7187,Autonomous vehicles could reduce traffic fatal...
9,724227040,1,0.6412,Driverless cars are not worth the risk. Don't...
10,724227041,3,0.9184,"Driverless cars are now legal in Florida, Cali..."
11,724227610,3,1.0000,Audi is the first carmaker to get a license fr...


In [54]:
from textblob.classifiers import NaiveBayesClassifier

In [55]:
subset = train_set[['text', 'sentiment']]
tuples = [tuple(x) for x in subset.values]

train_set[['text']][0:10]
subset[['text']]
len(tuples)

5009

In [56]:

#all_words =subset[['text']] 

all_words=toktok.tokenize(subset[['text']] )

#all_words
#all_words = set(word.lower() for passage in tuples for word in toktok.tokenize(passage[0]))
t = [({word: (word in (x[0])) for word in all_words}, x[1]) for x in tuples]
len(t)

training_set = t[:3500]
testing_set =  t[3500:]

In [59]:


classifier = nltk.NaiveBayesClassifier.train(training_set)
classifier.show_most_informative_features()

print("Naive Bayes Classifier accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

##SVC_classifier = SklearnClassifier(SVC())
##SVC_classifier.train(training_set)
##print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)



#print("voted_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100,"and SGD Classifier" (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

Most Informative Features
                    cool = True                5 : 3      =     48.8 : 1.0
                    seem = True                1 : 3      =     41.0 : 1.0
                    Damn = True                1 : 3      =     41.0 : 1.0
                    Fuck = True                1 : 3      =     41.0 : 1.0
                  reduce = True                5 : 3      =     22.8 : 1.0
                    used = True                1 : 3      =     18.6 : 1.0
                  window = True                1 : 3      =     17.6 : 1.0
                  places = True                5 : 3      =     16.3 : 1.0
                   given = True                1 : 3      =     13.7 : 1.0
                    mean = True                1 : 3      =     12.5 : 1.0
('Naive Bayes Classifier accuracy percent:', 40.75546719681908)
('MNB_classifier accuracy percent:', 50.76209410205435)
('BernoulliNB_classifier accuracy percent:', 42.080848243870115)
('LogisticRegression_classifier accurac